In [81]:
'''
这一栏是为了拿维州的全部postcodes
'''
import pandas as pd
# Defining of the dataframe
# built-in imports
import re
from json import dump

from collections import defaultdict

# user packages
from bs4 import BeautifulSoup, SoupStrainer
from urllib.request import urlopen
import requests
df = pd.DataFrame(columns=['postcode'])
headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
table = BeautifulSoup(requests.get('https://www.worldpostalcodes.org/l1/en/au/australia/list/r1/list-of-postcodes-in-victoria',headers = headers).text, "html.parser")
# Collecting Ddata
for row in table.find_all('tr'):    
    # Find all data for each column
    columns = row.find_all('td')
    
    if(columns != []):
        postcode = columns[0].text.strip()

        df = df.append({'postcode':postcode}, ignore_index=True)    
postcodes = df['postcode'].tolist()
postcodes = [int(x) for x in postcodes if x]

In [148]:
url_links = {}

674

In [149]:
"""
这一栏和下一栏不要跑，结果已经保存到了scrapping_url上，可以通过直接run 209 栏来直接读取
"""
# constants

postcode_list = postcodes # All the postcode in the Victoria

# begin code
# generate list of urls to visit
for postcode in postcode_list[674:]:
    search_postcode = f"https://www.oldlistings.com.au/search/postcode?id={postcode}&type=rent"
    bs_object = BeautifulSoup(requests.get(search_postcode,headers = headers).text, "html.parser")
    if bs_object.find("p", {"class": "sub-page-h1"}).text == "Suburb Not Found":
        continue
    else:
        if bs_object.find("p",{"class":"sub-page-h1"}).text != "Postcode Search":
            url_links[search_postcode] = postcode
        else:
            for url in bs_object.find("section",{"class":"page-wrap grid-container"}) \
                                .find("section",{"class":"grid-100"}) \
                                .findAll("a"):
                url_links["https://www.oldlistings.com.au"+url["href"]] = postcode
                

In [204]:
# Storing the url we need to scape 
import csv
with open('scrapping_url.csv', 'w') as f:
    for key in url_links.keys():
        f.write("%s, %s\n" % (key, url_links[key]))

In [209]:
# Starting here
# 文件在google drive
'''
这个东西是所有需要爬取信息的网站
'''
import pandas as pd
url_links = pd.read_csv("scrapping_url.csv",names=["links", "postcode"])

In [210]:
url_links[links]

,links,postcode
0,https://www.oldlistings.com.au/real-estate/VIC...,3000
1,https://www.oldlistings.com.au/real-estate/VIC...,3000
2,https://www.oldlistings.com.au/search/postcode...,3002
3,https://www.oldlistings.com.au/search/postcode...,3003
4,https://www.oldlistings.com.au/search/postcode...,3004
...,...,...
2413,https://www.oldlistings.com.au/real-estate/VIC...,3995
2414,https://www.oldlistings.com.au/real-estate/VIC...,3995
2415,https://www.oldlistings.com.au/real-estate/VIC...,3996
2416,https://www.oldlistings.com.au/real-estate/VIC...,3996


In [225]:
import random
# 随机header
user_agents =  ['Mozilla/5.0 (Windows NT 6.1; rv:2.0.1) Gecko/20100101 Firefox/4.0.1',
                'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50',
                'Opera/9.80 (Windows NT 6.1; U; en) Presto/2.8.131 Version/11.11',
               'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36',
               'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.1; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; InfoPath.3)']   
local_ip = '192.168.86.71:40000' # 不用改

In [242]:
# funtion 更改当前被ban的ip
def changeIP(current_ip):
    url = "https://free-proxy-list.net/"
    headers = {'User-Agent':random.choice(user_agents)} 
    bs_object = BeautifulSoup(requests.get(url,headers = headers).text, "html.parser") 
    for tr in bs_object.find("tbody").findAll("tr"):
        ip_information = tr.findAll("td")
        proxy = ip_information[0].text + ":" + ip_information[1].text
        if ip_information[4].text == "elite proxy" and proxy != current_ip:
            break
    return proxy

In [165]:
# for each url, scrape some basic metadata
'''
这里是正式开始爬取信息的地方，然后3168以前的邮编已经爬过了，可以尝试爬后面的
'''

from collections import defaultdict

property_metadata = defaultdict(dict)
current_ip = changeIP(local_ip)
for row,index in url_links.iterrows():
    # sleep for 0-1 seconds
    time.sleep(random.random())
    
    headers = {'User-Agent':random.choice(user_agents)}  
    
    # if 403 happened, changed current ip address 
    bs_object = BeautifulSoup(requests.get(row["links"],headers = headers, proxies={"http": current_ip, "https": current_ip}).text, "html.parser") 
    if bs_object.find("head").find("title").text == "403 Forbidden":
        current_ip = changeIP(current_ip)
        bs_object = BeautifulSoup(requests.get(row["links"],headers = headers, proxies={"http": current_ip, "https": current_ip}).text, "html.parser")
        
        
    postcodes = row["postcode"]
    bs_element = []
    if bs_object.find_all('div', {'class':'property odd clearfix'}) is not None: 
        bs_element.append(bs_object.find_all('div', {'class':'property odd clearfix'}))
    
    if bs_object.find_all('div', {'class':'property even clearfix'}) is not None:
        bs_element.append(bs_object.find_all('div', {'class':'property even clearfix'}))
    
    if len(bs_element) > 0:    
        for bs in bs_element:     
            for prop in bs:
                # looks for the header class to get property name
                name = prop \
                    .find("h2", {"class": "address"}) \
                    .text
                property_metadata[name]['name'] = prop \
                    .find("h2", {"class": "address"}) \
                    .text
                property_metadata[name]['postcode'] = postcodes

                cost = []     
                for li in prop.find("section", {"class": "grid-100 historical-price"}).find("ul"):              
                    cost_text = li.text
                    cost.append(cost_text)
                if " " in cost:
                    cost.remove(" ")
                property_metadata[name]['cost_text'] = cost

                if prop.find("p",{"class":"property-meta type"}) is not None:
                    property_metadata[name]['property_type'] = prop.find("p",{"class":"property-meta type"}).text

                if prop.find("p",{"class":"property-meta bed"}) is not None:
                    property_metadata[name]['bed'] = prop.find("p",{"class":"property-meta bed"}).text

                if prop.find("p",{"class":"property-meta bath"}) is not None:
                    property_metadata[name]['bath'] = prop.find("p",{"class":"property-meta bath"}).text

                if prop.find("p",{"class":"property-meta car"}) is not None:
                    property_metadata[name]['car'] = prop.find("p",{"class":"property-meta car"}).text

                property_metadata[name]['latitude'] = prop["data-lat"]  
                property_metadata[name]['longitude'] = prop["data-lng"]

# output to example json in data/raw/

with open('unfiltered.json', 'w') as f:
    dump(property_metadata, f)

In [1]:
import pandas as pd
df_raw = pd.read_json('../data/raw/3000-3168.json')
df_raw = df_raw.T
df_raw = df_raw.reset_index(drop=True)
df_raw

,name,postcode,cost_text,property_type,bed,bath,latitude,longitude,car
0,"609/1 BOUVERIE STREET, CARLTON SOUTH",3000,"[November 2021$500 per week, , September 2020...",Category : Unit/apmt,Bed : 3,Bath : 2,-37.8061001,144.96122001,NaN
1,"507/22 ORR STREET, CARLTON SOUTH",3000,"[June 2021$350 per week, , June 2021$390 per ...",Category : Unit/apmt,Bed : 2,Bath : 2,-37.80634001,144.96535001,NaN
2,"112/1 QUEENSBERRY STREET, CARLTON SOUTH",3000,"[April 2021$795 per week, , April 2021$645 pe...",NaN,Bed : 2,Bath : 2,-37.80541001,144.96875001,Car : 1
3,"3203/488 SWANSTON STREET, CARLTON SOUTH",3000,"[October 2020$430 per week, ]",NaN,Bed : 2,Bath : 1,-37.80558001,144.96354001,NaN
4,"506/1 BOUVERIE STREET, CARLTON SOUTH",3000,"[February 2019$420 per week, , May 2017$410 p...",Category : Unit/apmt,Bed : 1,Bath : 1,-37.8060974,144.9612201,NaN
...,...,...,...,...,...,...,...,...,...
15890,"1B/5 ACACIA PLACE, NOTTING HILL",3168,"[July 2022$330 per week, , January 2021$330 p...",Category : Unit/apmt,Bed : 1,Bath : 1,-37.90093001,145.13453001,Car : 1
15891,"G03/662-678 BLACKBURN ROAD, NOTTING HILL",3168,"[July 2022$390, , June 2022$350 p.w., , June...",Category : Unit/apmt,Bed : 2,Bath : 2,-37.9064001,145.14303001,Car : 2
15892,"103/416-420 FERNTREE GULLY ROAD, NOTTING HILL",3168,"[July 2022$395 per week, , April 2022$395 per...",Category : Unit/apmt,Bed : 2,Bath : 1,-37.901733,145.144169,Car : 1
15893,"G42, 658-660 BLACKBURN ROAD, NOTTING HILL",3168,"[July 2022$350, ]",Category : House,NaN,NaN,-37.90476001,145.1426001,Car : 1


In [10]:
df_raw_1 = pd.read_csv('../data/raw/row_852_1252.csv')
df_raw_1 = df_raw_1.T
df_raw_1 = df_raw_1.reset_index(drop=True)
df_raw_1

,0,1,2,3,4,5,6,7,8
0,name,postcode,cost_text,property_type,bed,bath,car,latitude,longitude
1,"2/8 DENNEYS STREET WARRNAMBOOL, WARRNAMBOOL WEST",3280,"['August 2010', ' ']",Category : Unit,Bed : 1,Bath : 1,Car : 1,-38.3846770,142.4969750
2,"21 MURRAY STREET WARRNAMBOOL, WARRNAMBOOL WEST",3280,"['August 2010', ' ']",Category : House,Bed : 3,Bath : 2,Car : 1,-38.3764160,142.4734010
3,"20 HICKFORD PDE WARRNAMBOOL, WARRNAMBOOL WEST",3280,"['August 2010', ' ']",Category : House,Bed : 4,Bath : 1,Car : 1,-38.3970830,142.5072400
4,"2/29 GLADSTONE STREET WARRNAMBOOL, WARRNAMBOOL...",3280,"['August 2010', ' ']",Category : House,Bed : 2,Bath : 1,Car : 1,-38.3933100,142.5046860
...,...,...,...,...,...,...,...,...,...
1535,"LOT 2, 49 WALLACE STREET, MEREDITH",3333,"['March 2013Contact', ' ', 'February 2013$320 ...",Category : House,Bed : 3,Bath : 2,Car : 2,-37.8512030,144.0777840
1536,"5490 MIDLAND HWY, MEREDITH",3333,"['May 2012$240 per week', ' ']",Category : House,Bed : 3,Bath : 1,NaN,-37.8281582,144.0516811
1537,"7 BARKERS LANE, ELAINE",3334,"['May 2019$350 per week', ' ']",Category : House,Bed : 4,Bath : 1,Car : 1,-37.73631900,144.06052500
1538,"324 ELAINE-EGERTON ROAD, ELAINE",3334,"['March 2015$350 p.w.', ' ']",NaN,Bed : 4,Bath : 2,Car : 2,-37.7580500,144.0515530
